In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 
cwd = "..\\..\\"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"
cwd_Images_Results = cwd + "\\Results"

In [3]:
df_HI = pd.read_excel(cwd + "\\Results\\Final (101 Sites) HI, Roman, Info.xlsx", sheet_name="Homogeneity", header = 1)
df_Roman = pd.read_excel(cwd + "\\Results\\Final (101 Sites) HI, Roman, Info.xlsx", sheet_name="Representativeness")
df = pd.merge(df_Roman,df_HI[['Site','CV 900']],on='Site')
df = df[['Site','Spatial Representativeness','RAW Score','ST Score','Roman Metrics','CV 900']].sort_values(['Site','Spatial Representativeness']).reset_index(drop = True)
df.head()

,Site,Spatial Representativeness,RAW Score,ST Score,Roman Metrics,CV 900
0,AT-Mmg,100-300,9.436081,0.676006,RAW,0.065007
1,AT-Mmg,100-600,27.656817,0.433034,RAW,0.065007
2,AT-Mmg,100-900,5.195374,0.259116,RAW,0.065007
3,AT-Mmg,300-600,13.563862,0.734345,RAW,0.065007
4,AT-Mmg,300-900,3.173046,0.466587,RAW,0.065007


In [4]:
list_Sites = list(df['Site'].unique())
list_CV = list(df['CV 900'].unique())

In [5]:
list_HomoTest = []
list_RomanMetrics = []
list_Check = []
list_RAW = []
list_ST = []
for i in range(len(list_Sites)):
    temp_Site = list_Sites[i]
    temp_CV = list_CV[i]
    # Bad homogeneity
    if temp_CV > 0.25:
        list_HomoTest.append("Heterogeneity")
        temp_df = df[df['Site'] == temp_Site]
        temp_idMax_RAW = temp_df['RAW Score'].idxmax()
        temp_idMax_ST = temp_df['ST Score'].idxmax()
        # RAW Score + ST Score
        if temp_df['Roman Metrics'].values[0] == 'RAW or ST':
            list_RomanMetrics.append('RAW or ST')
            # Check 300-600 and 300-900, whether it has the highest ST and RAW Score
            if temp_df['Spatial Representativeness'][temp_idMax_ST] == '300-900' and temp_df['Spatial Representativeness'][temp_idMax_RAW] == '300-900':
                list_Check.append('300-900')
                list_ST.append(temp_df['ST Score'][temp_idMax_ST])
                list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
            elif temp_df['Spatial Representativeness'][temp_idMax_RAW] == '300-600' and temp_df['Spatial Representativeness'][temp_idMax_ST] == '300-600':
                list_Check.append('300-600')
                list_ST.append(temp_df['ST Score'][temp_idMax_ST])
                list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
            else:
                list_Check.append("Not Suitable for Validation")
                list_ST.append(None)
                list_RAW.append(None)
        # RAW Score
        else:
            list_RomanMetrics.append('RAW')
            # Check 300-900, whether it has the highest RAW Score
            if temp_df['Spatial Representativeness'][temp_idMax_RAW] == '300-900':
                list_Check.append('300-900')
                list_ST.append(None)
                list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
            elif temp_df['Spatial Representativeness'][temp_idMax_RAW] == '300-600':
                list_Check.append('300-600')
                list_ST.append(None)
                list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
            else:
                list_Check.append("Not Suitable for Validation")
                list_ST.append(None)
                list_RAW.append(None)
    # Good homogeneity
    else:
        list_HomoTest.append("Homogeneity")
        temp_df = df[df['Site'] == temp_Site]
        # RAW Score + ST Score
        if temp_df['Roman Metrics'].values[0] == 'RAW or ST':
            list_RomanMetrics.append('RAW or ST')
            temp_idMax_RAW = temp_df['RAW Score'].idxmax()
            temp_idMax_ST = temp_df['ST Score'].idxmax()
            # Check whether the highest ST and RAW Score belong to the same ROI
            if temp_idMax_RAW == temp_idMax_ST:
                temp_BestROI = temp_df['Spatial Representativeness'][temp_idMax_RAW]
                list_Check.append(temp_BestROI)
                list_ST.append(temp_df['ST Score'][temp_idMax_ST])
                list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
            else:
                if temp_idMax_RAW < temp_idMax_ST:
                    temp_BestROI_RAW = temp_df['Spatial Representativeness'][temp_idMax_RAW]
                    list_Check.append(temp_BestROI_RAW)
                    list_ST.append(temp_df['ST Score'][temp_idMax_RAW])
                    list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
                else:
                    temp_BestROI_ST = temp_df['Spatial Representativeness'][temp_idMax_ST]
                    list_Check.append(temp_BestROI_ST)
                    list_ST.append(temp_df['ST Score'][temp_idMax_ST])
                    list_RAW.append(temp_df['RAW Score'][temp_idMax_ST])
        # RAW Score
        else:
            # Directly select the ROI with the highest RAW Score
            list_RomanMetrics.append('RAW')
            temp_idMax_RAW = temp_df['RAW Score'].idxmax()
            temp_BestROI = temp_df['Spatial Representativeness'][temp_idMax_RAW]
            list_Check.append(temp_BestROI)
            list_ST.append(None)
            list_RAW.append(temp_df['RAW Score'][temp_idMax_RAW])
list_Check

['100-600',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 '300-600',
 '300-600',
 '300-600',
 '300-600',
 '300-600',
 '300-900',
 '100-300',
 '300-600',
 'Not Suitable for Validation',
 '300-600',
 '300-900',
 '100-300',
 '300-600',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 '300-600',
 'Not Suitable for Validation',
 '300-600',
 '100-300',
 'Not Suitable for Validation',
 '300-600',
 '300-600',
 '300-600',
 '300-600',
 'Not Suitable for Validation',
 '300-600',
 '100-900',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 '300-600',
 '300-600',
 '300-900',
 '300-600',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 '100-300',
 '100-300',
 '100-300',
 'Not Suitable for Validation',
 '300-600',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 '300-600',
 '100-300',
 'Not Suitable for Validation',
 'Not Suitable for Validation',
 '300-900',
 '100-300',
 '300-600',
 '30

In [6]:
df_Final = pd.DataFrame({
    "Site": list_Sites,
    "Homogeneity Test": list_HomoTest,
    "Roman Metrics": list_RomanMetrics,
    "Best ROI": list_Check,
    "RAW Score": list_RAW,
    "ST Score": list_ST
})
df_Final

,Site,Homogeneity Test,Roman Metrics,Best ROI,RAW Score,ST Score
0,AT-Mmg,Homogeneity,RAW,100-600,27.656817,NaN
1,ATGE,Heterogeneity,RAW or ST,Not Suitable for Validation,NaN,NaN
2,ATLAS-Mohammed V,Heterogeneity,RAW or ST,Not Suitable for Validation,NaN,NaN
3,ATLAS-Mohammed V New,Heterogeneity,RAW or ST,300-600,51.318989,0.799848
4,BASP,Heterogeneity,RAW or ST,300-600,0.362665,0.657212
...,...,...,...,...,...,...
96,US-OPE,Heterogeneity,RAW,300-600,1.676705,NaN
97,US-SERC,Homogeneity,RAW or ST,100-300,4.158097,1.059717
98,US-SERC New,Homogeneity,RAW,300-600,1.935230,NaN
99,Utqiagvik,Heterogeneity,RAW or ST,Not Suitable for Validation,NaN,NaN


In [7]:
df_Final.to_csv(cwd_Images_Results + "\\Best ROI Roberto.csv", index = False)